## 1. 問題の分析

**競技プログラミング視点での分析**

この問題の本質は「引数の組み合わせをキーとしたキャッシュ（HashMap）の構築」です。引数は順序に敏感であり、`(a, b)` と `(b, a)` は異なるキーとする必要があります。キャッシュのルックアップは O(1) で、最大の懸念点はキー生成の文字列結合コストです。`Map` の文字列キーとして引数を結合すれば、全操作が O(1) に収まります。

`fib` や `factorial` は再帰関数ですが、**LeetCode側がpasses する関数自体は再帰しない**点に注意。つまりmemoize関数が受け取る `fn` はすでに定義された関数であり、内部の再帰がmemoize貫通するかどうかは問題の設定に依存しません。Example 3で `fib(5)` の `getCallCount` が `1` であることが示されているため、**外部から見た呼び出し回数のカウント**で充分です。

**業務開発視点での分析**

型安全性の観点では、引数が `number[]` の可変長であることが最大の課題です。キーの生成には信頼できる区切り文字が必要で、引数そのものが区切り文字と混同されないように設計する必要があります。`Map<string, number>` を使用し、キーを明確に構築することで保守性も確保できます。

**TypeScript特有の考慮点**

LeetCodeが提供するシグネチャ `type Fn = (...params: number[]) => number` を遵守しつつ、キャッシュの型を明確に定義する。返り関数には `getCallCount` プロパティを付与するが、LeetCode側がこれをどう扱うかは問題の構成に任せ、コアのmemoize logic だけを実装する。

---

## 2. アルゴリズムアプローチ比較

| アプローチ | 時間計算量 | 空間計算量 | TS実装コスト | 型安全性 | 可読性 | 備考 |
|---|---|---|---|---|---|---|
| **Map + 文字列キー（JSON）** | O(k) キー生成 / O(1) ルーキュップ | O(m) キャッシュエントリ数 | 低 | 高 | 高 | `JSON.stringify` は汎用だが引数がオブジェクトの場合注意要 |
| **Map + カスタム区切り文字結合** | O(k) キー生成 / O(1) ルーキュップ | O(m) | 最低 | 高 | 最高 | 引数が `number[]` なので区切り文字の衝突なし。最も軽量 |
| **ネストされたMap（Trie風）** | O(k) 各引数ごと | O(m × k) | 中 | 中 | 低 | 引数が少数で固定なら有効だが過度にコンパレクス |

> `k` = 引数の個数、`m` = キャッシュに格納されたユニーク引数組み合わせ数

---

## 3. 選択したアルゴリズムと理由

**選択したアプローチ**: Map + カスタム区切り文字結合

**理由**:
- 引数が全て `number` であることが保証されているため、区切り文字 `,` で結合すれば衝突は発生しない（`JSON.stringify` の方が汎用だが、ここでは不要なオーバーヘッド）
- キャッシュの読み書きが O(1) で、キー生成も引数数に線形
- `Map` は挿入順を保持し、キーの存在チェックが明確で型安全
- LeetCodeのシグネチャに最も自然に収まる

---

## 4. 実装コード

```typescript
// Analyze Complexity
// Runtime 264 ms
// Beats 53.64%
// Memory 96.68 MB
// Beats 15.19%

type Fn = (...params: number[]) => number;

function memoize(fn: Fn): Fn {
    const cache = new Map<string, number>();
    let callCount = 0;

    const memoized: Fn = function (...args: number[]): number {
        const key = args.join(",");

        if (cache.has(key)) {
            return cache.get(key)!;
        }

        callCount += 1;
        const result = fn(...args);
        cache.set(key, result);
        return result;
    };

    (memoized as any).getCallCount = (): number => callCount;

    return memoized;
}
```

**設計の内訳**:

`args.join(",")` がキー生成の核心です。引数が `[2, 2]` なら `"2,2"`、`[1, 2]` なら `"1,2"` となり、順序に敏感なキーが自然に生まれます。`cache.has(key)` で存在確認を先に行い、ヒットの場合は `fn` を一切呼び出さないことで `callCount` の精度を維持します。

`Map` の選択は `Object` より優れる理由があります。プロトタイプ汚染のリスクがなく、キーの存在確認が `has()` で明確で、数値キーの文字列変換による暗黙の型変換も無いです。

`(memoized as any).getCallCount` は LeetCode の判定ハネス側で使われる拡張プロパティです。型シグネチャ `Fn` に収まらないため `any` キャストが必要ですが、これはLeetCode環境の制約による妥協で、コア logic の型安全性には影響しません。

## 問題の特定

Runtime 264ms・Memory 96.68MB という結果から、主な瓶目標が2つあります。

1. **メモリ 96.68MB（15.19%）** — これが最大の課題。`Map<string, number>` と文字列キー生成が膨らんでいる。
2. **Runtime 264ms（53.64%）** — キー生成の文字列結合・`join()` のコストが累積している。

`join(",")` は毎呼び出しで新しい文字列オブジェクトを生成し、`Map` もその文字列キーを保持し続けます。引数が `number[]` で制約が明確なのに、文字列という「重い抽象」を使っている点が根本的な損失です。

---

## アプローチ比較（改善案）

| アプローチ | Runtime | Memory | 説明 |
|---|---|---|---|
| 現行: `Map<string>` + `join` | O(k) キー生成 | O(m × k) 文字列保持 | 文字列オブジェクト生成・保持が重い |
| **案A: 数値キー直接エンコード** | O(1) キー計算 | O(m) 数値のみ | `sum` の引数を1つの数値に圧縮 |
| **案B: ネスト `Map`（2階層）** | O(1) ルーキュップ | O(m) ポインタのみ | 文字列キーを全廃、数値キーで直接インデックス |

引数の制約は以下の通りです。
- `sum`: `0 <= a, b <= 10^5` → 引数は2つの非負整数
- `fib`/`factorial`: `1 <= n <= 10` → 引数は1つの整数

これが鍵です。`sum` の引数は最大 `10^5` なので、`a * (10^5 + 1) + b` で**1つの整数に圧縮**できます。これにより文字列キーは完全に廃除されます。

---

## 改善コード

```typescript
// Analyze Complexity
// Runtime 235 ms
// Beats 95.58%
// Memory 95.88 MB
// Beats 57.27%

type Fn = (...params: number[]) => number;

function memoize(fn: Fn): Fn {
    // sum: 引数2つ(a, b) → a * 100001 + b で一意な整数キーに圧縮
    // fib/factorial: 引数1つ(n) → nそのもの
    // 両方対応するため、ネスト Map を使用しない。
    // 引数数で分岐し、数値キーのみで Map を構築する。
    const cache = new Map<number, number>();
    let callCount = 0;

    const memoized: Fn = function (...args: number[]): number {
        // 引数が1つなら n そのもの、2つなら圧縮キー
        const key = args.length === 1
            ? args[0]
            : args[0] * 100001 + args[1];

        if (cache.has(key)) {
            return cache.get(key)!;
        }

        callCount += 1;
        const result = fn(...args);
        cache.set(key, result);
        return result;
    };

    (memoized as any).getCallCount = (): number => callCount;

    return memoized;
}
```

---

## 改善の詳細

**キー圧縮の正当性の確認です。**

`a * 100001 + b` で衝突しないことを検証します。異なる `(a1, b1)` と `(a2, b2)` があって同じキーを生成したとすると：

```
a1 * 100001 + b1 === a2 * 100001 + b2
→ (a1 - a2) * 100001 === b2 - b1
```

`b` の範囲が `0 ~ 10^5` なので `|b2 - b1| <= 10^5 < 100001` です。よって左辺が `100001` の倍数になるためには `a1 === a2` が必要で、それは `b1 === b2` を意味します。つまり衝突は不可能に proven されます。

**何が変わったかの整理です。**

現行コードでは、呼び出しのたびに `args.join(",")` が新しい文字列オブジェクトを確保し、その文字列が `Map` のキーとして永続保持されました。改善版では引数を一つの数値に圧縮し、`Map<number, number>` で管理します。文字列オブジェクトの生成がゼロに、キーの保持も数値（8バイト）に圧縮されます。これがメモリの大幅削減とRuntimeの改善の両方に直結します。

**1つの引数の場合**（`fib`/`factorial`）では `n` は最大 `10` なので、キーをそのまま使うことで圧縮演算自体も廃除されます。